In [1]:
import pandas as pd

In [48]:
df = pd.read_csv('data/AllData.csv')

In [49]:
features_cols = ['GridRows', 'GridColumns', 'NumOfAgents', 'NumOfObstacles', 'BranchingFactor', 'ObstacleDensity',
            'AvgDistanceToGoal', 'MaxDistanceToGoal', 'MinDistanceToGoal', 'AvgStartDistances', 'AvgGoalDistances',
            'PointsAtSPRatio']
runtime_cols = list(df.filter(like="Runtime"))
relev_cols = runtime_cols + features_cols + ['Y']

In [50]:
runtime_cols

['A*+OD+ID Runtime',
 'MA-CBS-Global-10/(A*+OD/SIC) choosing the first conflict in CBS nodes Runtime',
 'MA-CBS-Local-10/(single:A*/SIC multi:A*+OD/SIC) choosing the first conflict in CBS nodes Runtime',
 'Basic-CBS/(A*+OD/SIC) choosing the first conflict in CBS nodes Runtime',
 'MA-CBS-Global-10/(EPEA*/SIC) choosing the first conflict in CBS nodes Runtime',
 'MA-CBS-Local-10/(single:A*/SIC multi:EPEA*/SIC) choosing the first conflict in CBS nodes Runtime',
 'Basic-CBS/(A*/SIC) choosing cardinal conflicts using lookahead Runtime',
 'Basic-CBS/(A*/SIC)+ID Runtime',
 'ICTS 2RE Runtime']

In [51]:
Y = df[runtime_cols].idxmin(axis=1)
df['Y'] = Y

In [52]:
def runtime_to_success(col):
    splitted = col.split()
    splitted[-1] = "Success"
    return " ".join(splitted)

runtime_to_success("Y Runtime")

'Y Success'

In [53]:
df['Y Success'] = df.apply(lambda x:x[runtime_to_success(x['Y'])], axis=1)

In [54]:
df['Y Runtime'] = df.apply(lambda x:x[x['Y']], axis=1)

In [55]:
df['Y Runtime']

0            0.0619
1            0.0824
2            0.0949
3            0.6920
4            0.5637
5            0.1618
6            0.9018
7            7.7158
8           23.9858
9            1.6387
10           2.1108
11          16.9749
12           8.5879
13           9.6028
14          13.2872
15           0.0609
16           0.0743
17           0.1093
18           0.9492
19           0.1002
20           0.1396
21          11.3411
22           9.0184
23           1.0061
24           5.9968
25           2.9409
26          11.6477
27           7.0104
28           4.3265
29           5.0387
           ...     
3430    300001.0477
3431       344.2759
3432     11305.4896
3433    232497.6007
3434     76474.9938
3435     91414.1211
3436    300001.6802
3437    135715.2215
3438      2172.9895
3439    300001.1623
3440       154.4607
3441        78.3952
3442        18.5793
3443        32.1162
3444        97.9156
3445        32.9038
3446       676.4372
3447        43.3181
3448      1139.8572


In [56]:
df['Y'].value_counts()

MA-CBS-Global-10/(EPEA*/SIC) choosing the first conflict in CBS nodes Runtime                       755
Basic-CBS/(A*/SIC) choosing cardinal conflicts using lookahead Runtime                              573
MA-CBS-Local-10/(single:A*/SIC multi:EPEA*/SIC) choosing the first conflict in CBS nodes Runtime    539
A*+OD+ID Runtime                                                                                    395
MA-CBS-Local-10/(single:A*/SIC multi:A*+OD/SIC) choosing the first conflict in CBS nodes Runtime    370
MA-CBS-Global-10/(A*+OD/SIC) choosing the first conflict in CBS nodes Runtime                       321
Basic-CBS/(A*/SIC)+ID Runtime                                                                       321
Basic-CBS/(A*+OD/SIC) choosing the first conflict in CBS nodes Runtime                              182
ICTS 2RE Runtime                                                                                      4
Name: Y, dtype: int64

## Add Sparsity feature 

In [57]:
df['GridSize'] = df['GridRows'] * df['GridColumns']

In [58]:
df['Sparsity'] = df.apply(lambda x: x['NumOfAgents']/(x['GridSize']-x['NumOfObstacles']),axis=1)

In [59]:
features_cols.append('GridSize')
features_cols.append('Sparsity')


In [60]:
features_cols.append('Y')
df[features_cols].corr()

,GridRows,GridColumns,NumOfAgents,NumOfObstacles,BranchingFactor,ObstacleDensity,AvgDistanceToGoal,MaxDistanceToGoal,MinDistanceToGoal,AvgStartDistances,AvgGoalDistances,PointsAtSPRatio,GridSize,Sparsity
GridRows,1.000000,1.000000,-0.156633,0.741861,0.000596,0.019507,0.937977,0.910771,0.701494,0.866240,0.866639,-0.604342,0.966714,-0.489521
GridColumns,1.000000,1.000000,-0.156633,0.741861,0.000596,0.019507,0.937977,0.910771,0.701494,0.866240,0.866639,-0.604342,0.966714,-0.489521
NumOfAgents,-0.156633,-0.156633,1.000000,-0.172761,0.382752,-0.025351,-0.145026,0.018586,-0.356525,0.074557,0.075870,0.674187,-0.221822,0.143032
NumOfObstacles,0.741861,0.741861,-0.172761,1.000000,-0.022311,0.403654,0.693205,0.663647,0.511844,0.637986,0.650474,-0.431318,0.767086,-0.275584
BranchingFactor,0.000596,0.000596,0.382752,-0.022311,1.000000,-0.009609,-0.000977,0.042795,-0.065872,0.046287,0.050269,0.173279,-0.025352,0.023109
ObstacleDensity,0.019507,0.019507,-0.025351,0.403654,-0.009609,1.000000,0.047019,0.044819,-0.004634,0.042141,0.059845,-0.123578,0.017141,0.043461
AvgDistanceToGoal,0.937977,0.937977,-0.145026,0.693205,-0.000977,0.047019,1.000000,0.941071,0.783530,0.844592,0.841542,-0.559638,0.902071,-0.465160
MaxDistanceToGoal,0.910771,0.910771,0.018586,0.663647,0.042795,0.044819,0.941071,1.000000,0.590421,0.899972,0.898901,-0.455269,0.861666,-0.448322
MinDistanceToGoal,0.701494,0.701494,-0.356525,0.511844,-0.065872,-0.004634,0.783530,0.590421,1.000000,0.490642,0.473756,-0.556456,0.698529,-0.352339
AvgStartDistances,0.866240,0.866240,0.074557,0.637986,0.046287,0.042141,0.844592,0.899972,0.490642,1.000000,0.918090,-0.388959,0.823494,-0.413017


## Save labelled data to file

In [61]:
df.to_csv("data/AllData-labelled.csv")